In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import RFE,RFECV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import  metrics
from sklearn.metrics import roc_curve, plot_roc_curve,auc,make_scorer,accuracy_score,roc_auc_score,confusion_matrix, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier as MLP
import shap
import pdpbox

In [ ]:
Set2 = sns.color_palette('Set2')
Set2

# 筛选影像特征

## 预处理平扫数据

In [ ]:
# 读取数据集

df_cli = pd.read_excel('./immunotherapy_112_V4.xlsx')
df_cli = df_cli.set_index('影像号',drop=True)

def f(x):
    return df_cli.loc[x, 'Label_0 (DCB=0 PFS>6,NDB=1 PFS≤6)']
df_pre = pd.read_csv('./precontrast_features.csv')
X_pre = df_pre.iloc[:,1:].values
Y_pre = df_pre.patient_id.map(f)

In [ ]:
# 标签数量统计

pd.value_counts(Y_pre)

In [ ]:
# 数据扩充

ros = RandomOverSampler(random_state=42)
X_pre_up, Y_pre_up = ros.fit_resample(X_pre, Y_pre)

pd.value_counts(Y_pre_up)

In [ ]:
# 特征筛选
X_train, X_test, Y_train, Y_test = train_test_split(X_pre_up, Y_pre_up, test_size=0.3, shuffle=True, random_state=42)
select = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42),
             n_features_to_select=15,
             step=0.02
            )
select.fit(X_train, Y_train)
X_train_selected = select.transform(X_train)

estimator_pre = select.estimator_  # 用于计算平扫rad score的随机森林模型

features_pre = df_pre.iloc[:, 1:].columns.values[select.get_support()]  # 筛选的特征名

features_pre_imp = select.estimator_.feature_importances_  # 筛选的特征重要性

In [ ]:
# 绘制特征重要性图
# 修改字体
from matplotlib import rcParams
params={'font.family':'sans-serif',
        'font.sans-serif':'Arial',
        'font.weight':'normal',
        'font.size': 15
        }
rcParams.update(params)
tb_preimp = pd.DataFrame({'x':features_pre, 'y': features_pre_imp})
tb_preimp = tb_preimp.sort_values('y', ascending=False)
fig = plt.figure(figsize=(8,6))
sns.barplot(x="y", y="x", data=tb_preimp, color=Set2[2])
plt.xlabel('Feature importance')
plt.ylabel('')
# plt.savefig('Feature_importance_precontrast.pdf', bbox_inches = 'tight')
# plt.savefig('Feature_importance_precontrast.png', bbox_inches = 'tight', dpi=600)

In [ ]:
## Metric

# acc
acc_pre = select.estimator_.score(X_test[:,select.get_support()], Y_test)

# auc
proba_pre = select.estimator_.predict_proba(X_test[:,select.get_support()])[:,1]
auc_pre = roc_auc_score(Y_test, proba_pre)

tn, fp, fn, tp = confusion_matrix(y_true=Y_test, y_pred=select.estimator_.predict(X_test[:,select.get_support()])).ravel()
# sensitivity
sensitivity_pre = tp / (tp + fn)

# specificity
specificity_pre = tn / (tn + fp)

print('ACC: {:.3}, AUC: {:.3}, Sensitivity:{:.3f}, Specificity: {:.3}'.format(acc_pre, auc_pre, sensitivity_pre, specificity_pre))

## 预处理增强数据

In [ ]:
# 读取数据集

df_cli = pd.read_excel('./immunotherapy_112_V4.xlsx')
df_cli = df_cli.set_index('影像号',drop=True)

def f(x):
    return df_cli.loc[x, 'Label_0 (DCB=0 PFS>6,NDB=1 PFS≤6)']
df_pos = pd.read_csv('./postcontrast_features.csv')
X_pos = df_pos.iloc[:,1:].values
Y_pos = df_pos.patient_id.map(f)

In [ ]:
# 标签数量统计

pd.value_counts(Y_pos)

In [ ]:
# 数据扩充

ros = RandomOverSampler(random_state=42)
X_pos_up, Y_pos_up = ros.fit_resample(X_pos, Y_pos)

pd.value_counts(Y_pos_up)

In [ ]:
# 特征筛选
X_train, X_test, Y_train, Y_test = train_test_split(X_pos_up, Y_pos_up, test_size=0.3, shuffle=True, random_state=42)
select = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42),
             n_features_to_select=15,
             step=0.02
            )
select.fit(X_train, Y_train)
X_train_selected = select.transform(X_train)

estimator_pos = select.estimator_  # 用于计算平扫rad score的随机森林模型

features_pos = df_pos.iloc[:, 1:].columns.values[select.get_support()]  # 筛选的特征名

features_pos_imp = select.estimator_.feature_importances_  # 筛选的特征重要性

In [ ]:
# 绘制特征重要性图
# 修改字体
from matplotlib import rcParams
params={'font.family':'sans-serif',
        'font.sans-serif':'Arial',
        'font.weight':'normal',
        'font.size': 15
        }

tb_posimp = pd.DataFrame({'x':features_pos, 'y': features_pos_imp})
tb_posimp = tb_posimp.sort_values('y', ascending=False)
fig = plt.figure(figsize=(8,6))
sns.barplot(x="y", y="x", data=tb_posimp, color=Set2[2])
plt.xlabel('Feature importance')
plt.ylabel('')
# plt.savefig('Feature_importance_poscontrast.pdf', bbox_inches = 'tight')
# plt.savefig('Feature_importance_poscontrast.png', bbox_inches = 'tight', dpi=600)

In [ ]:
# Metric

# acc
acc_pos = select.estimator_.score(X_test[:,select.get_support()], Y_test)

# auc
proba_pos = select.estimator_.predict_proba(X_test[:,select.get_support()])[:,1]
auc_pos = roc_auc_score(Y_test, proba_pos)

tn, fp, fn, tp = confusion_matrix(y_true=Y_test, y_pred=select.estimator_.predict(X_test[:,select.get_support()])).ravel()
# sensitivity
sensitivity_pos = tp / (tp + fn)

# specificity
specificity_pos = tn / (tn + fp)

print('ACC: {:.3}, AUC: {:.3}, Sensitivity:{:.3f}, Specificity: {:.3}'.format(acc_pos, auc_pos, sensitivity_pos, specificity_pos))

# 筛选临床特征

In [ ]:
# 数据扩充
X_cli = df_cli.iloc[:, 4:].values
Y_cli = df_cli.loc[:, 'Label_0 (DCB=0 PFS>6,NDB=1 PFS≤6)']
ros = RandomOverSampler(random_state=42)
X_cli_up, Y_cli_up = ros.fit_resample(X_cli, Y_cli)

In [ ]:
# 特征筛选
X_train, X_test, Y_train, Y_test = train_test_split(X_cli_up, Y_cli_up, test_size=0.3, shuffle=True, random_state=42)
select = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42),
             n_features_to_select=5,
             step=0.02
            )
select.fit(X_train, Y_train)
X_train_selected = select.transform(X_train)

estimator_cli = select.estimator_  # 用于计算平扫rad score的随机森林模型

features_cli = df_cli.iloc[:, 4:].columns.values[select.get_support()]  # 筛选的特征名

features_cli_imp = select.estimator_.feature_importances_  # 筛选的特征重要性

In [ ]:
# 绘制特征重要性图
from matplotlib import rcParams
params={'font.family':'sans-serif',
        'font.sans-serif':'Arial',
        'font.weight':'normal',
        'font.size': 15
        }

# tb_cliimp = pd.DataFrame({'x':features_cli, 'y': features_cli_imp})
tb_cliimp = pd.DataFrame({'x':["BMI", "Hemoglobin", 'LDH', 'NLR', 'PLR'], 'y': features_cli_imp})
tb_cliimp = tb_cliimp.sort_values('y', ascending=False)
fig = plt.figure(figsize=(6,4))
sns.barplot(x="y", y="x", data=tb_cliimp, color=Set2[2])
plt.xlabel('Feature importance')
plt.ylabel('')
# plt.savefig('Feature_importance_cli.pdf', bbox_inches = 'tight')
# plt.savefig('Feature_importance_cli.png', bbox_inches = 'tight', dpi=600)

In [ ]:
# Metric

# acc
acc_cli = select.estimator_.score(X_test[:,select.get_support()], Y_test)

# auc
proba_cli = select.estimator_.predict_proba(X_test[:,select.get_support()])[:,1]
auc_cli = roc_auc_score(Y_test, proba_cli)

tn, fp, fn, tp = confusion_matrix(y_true=Y_test, y_pred=select.estimator_.predict(X_test[:,select.get_support()])).ravel()
# sensitivity
sensitivity_cli = tp / (tp + fn)

# specificity
specificity_cli = tn / (tn + fp)

print('ACC: {:.3}, AUC: {:.3}, Sensitivity:{:.3f}, Specificity: {:.3}'.format(acc_cli, auc_cli, sensitivity_cli, specificity_cli))

# 合并临床数据和radscore

In [ ]:
# 合并数据

df_merge = df_cli.loc[:, ['Label_0 (DCB=0 PFS>6,NDB=1 PFS≤6)'] + list(features_cli)]
df_merge = df_merge.sort_index()
df_merge = df_merge.reset_index(drop=True)
df_rad_pre = df_pre.sort_values('patient_id').loc[:, features_pre].reset_index(drop=True)
df_rad_pos = df_pos.sort_values('patient_id').loc[:, features_pos].reset_index(drop=True)
df_merge = pd.concat([df_merge, df_rad_pos, df_rad_pre], axis=1)

In [ ]:
# 数据扩充

X_merge = df_merge.iloc[:, 1:].values
Y_merge = df_merge.loc[:, 'Label_0 (DCB=0 PFS>6,NDB=1 PFS≤6)']
ros = RandomOverSampler(random_state=42)
X_merge_up, Y_merge_up = ros.fit_resample(X_merge, Y_merge)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_merge_up, Y_merge_up, test_size=0.3, shuffle=True, random_state=42)
RF = RandomForestClassifier(n_estimators=20, random_state=42)
RF.fit(X_train, Y_train)

# Metric
# acc
acc_merge = RF.score(X_test, Y_test)

# auc
proba_merge = RF.predict_proba(X_test)[:,1]
auc_merge = roc_auc_score(Y_test, proba_merge)

tn, fp, fn, tp = confusion_matrix(y_true=Y_test, y_pred=RF.predict(X_test)).ravel()
# sensitivity
sensitivity_merge = tp / (tp + fn)

# specificity
specificity_merge = tn / (tn + fp)

print('ACC: {:.3}, AUC: {:.3}, Sensitivity:{:.3f}, Specificity: {:.3}'.format(acc_merge, auc_merge, sensitivity_merge, specificity_merge))

In [ ]:
print('Pre   ACC: {:.3}, AUC: {:.3}, Sensitivity:{:.3f}, Specificity: {:.3}'.format(acc_pre, auc_pre, sensitivity_pre, specificity_pre))
print('Pos   ACC: {:.3}, AUC: {:.3}, Sensitivity:{:.3f}, Specificity: {:.3}'.format(acc_pos, auc_pos, sensitivity_pos, specificity_pos))
print('Clin  ACC: {:.3}, AUC: {:.3}, Sensitivity:{:.3f}, Specificity: {:.3}'.format(acc_cli, auc_cli, sensitivity_cli, specificity_cli))
print('Merge ACC: {:.3}, AUC: {:.3}, Sensitivity:{:.3f}, Specificity: {:.3}'.format(acc_merge, auc_merge, sensitivity_merge, specificity_merge))

# 构建影像特征的模型

采用多种机器学习方法，仅利用影像特征构建模型

In [ ]:
# 数据预处理

df_radiomic = pd.concat([df_rad_pos, df_rad_pre], axis=1)
X_radiomic = df_radiomic.values
Y_radiomic = Y_merge

ros = RandomOverSampler(random_state=42)
X_radiomic_up, Y_radiomic_up = ros.fit_resample(X_radiomic, Y_radiomic)

## 网格搜索

In [ ]:
# 网格搜索：随机森林
tuned_parameters = param_grid = {'n_estimators': [30, 50, 100, 150], 
                                 'max_features': [5, 10, 15, 20]}

#n_jobs =-1使用全部CPU并行多线程搜索
gs_rf = GridSearchCV(RandomForestClassifier(), 
                  tuned_parameters, 
                  refit = True, 
                  cv = 5,
                  scoring = 'roc_auc',
                  verbose = 1, 
                  n_jobs = -1)
gs_rf.fit(X_radiomic_up,Y_radiomic_up) #Run fit with all sets of parameters.
print('RF最优参数: ',gs_rf.best_params_)
print('RF最佳性能: ', gs_rf.best_score_)

In [ ]:
# 网格搜索：SVM
tuned_parameters = {'kernel': ['rbf', 'sigmoid', 'linear'], 
                     'C': [1, 10, 100]}

#n_jobs =-1使用全部CPU并行多线程搜索
gs_svm = GridSearchCV(SVC(), 
                  tuned_parameters, 
                  refit = True, 
                  cv = 5,
                  scoring = 'roc_auc',
                  verbose = 1, 
                  n_jobs = -1)
gs_svm.fit(X_radiomic_up,Y_radiomic_up) #Run fit with all sets of parameters.
print('SVM最优参数: ',gs_svm.best_params_)
print('SVM最佳性能: ', gs_svm.best_score_)

In [ ]:
# 网格搜索：HistGradientBoostingClassifier
tuned_parameters = {'max_leaf_nodes': [5, 10, 15, 20, 25,30], 
                     'min_samples_leaf': [5, 10, 15, 20, 25,30],
                    'max_iter': [30, 50, 100, 150]
                   }

#n_jobs =-1使用全部CPU并行多线程搜索
gs_hgbc = GridSearchCV(HistGradientBoostingClassifier(), 
                  tuned_parameters, 
                  refit = True, 
                  cv = 5,
                  scoring = 'roc_auc',
                  verbose = 1, 
                  n_jobs = -1)
gs_hgbc.fit(X_radiomic_up,Y_radiomic_up) #Run fit with all sets of parameters.
print('HGBC最优参数: ',gs_hgbc.best_params_)
print('HGBC最佳性能: ', gs_hgbc.best_score_)

In [ ]:
# 网格搜索：XGboost
tuned_parameters = {'n_estimators': [30, 50, 100, 150],
                     'learning_rate': [0.1, 0.01],
                    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11]
                   }

#n_jobs =-1使用全部CPU并行多线程搜索
gs_xgboost = GridSearchCV(xgb.XGBClassifier(), 
                  tuned_parameters, 
                  refit = True, 
                  cv = 5,
                  scoring = 'roc_auc',
                  verbose = 1, 
                  n_jobs = -1)
gs_xgboost.fit(X_radiomic_up,Y_radiomic_up) #Run fit with all sets of parameters.
print('XGboost最优参数: ',gs_xgboost.best_params_)
print('XGboost最佳性能: ', gs_xgboost.best_score_)

In [ ]:
tuned_parameters = {"hidden_layer_sizes": [(64,32,8), (32,16,8)],
                    "solver": ['adam', 'sgd', 'lbfgs'],
                    "max_iter": [50,100],
                    "batch_size":[8,16],
                    "alpha":[1e-3],
                    "activation":['logistic', 'tanh', 'relu']
                     }
gs_mlp = GridSearchCV(MLP(random_state=42), 
                  tuned_parameters, 
                  refit = True, 
                  cv = 5,
                  verbose = 1,
                  scoring = 'roc_auc',
                  n_jobs = -1)
gs_mlp.fit(X_radiomic_up,Y_radiomic_up) #Run fit with all sets of parameters.
print('MLP最优参数: ',gs_mlp.best_params_)
print('MLP最佳AUC: ', gs_mlp.best_score_)

## 模型训练

In [ ]:
##############################  Init Plot  ####################################
fig, ax = plt.subplots(figsize=(10,8))  # 用于绘制平均交叉验证
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey',
        label='Reference Line', alpha=.8)
ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="ROC curve with multiple machine learning methods")
df_out = pd.DataFrame(columns=['Method','Acc' ,'AUC', 'Spec', 'Sens'])


##############################  LR  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值

for train_index, test_index in kf.split(X=X_radiomic_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_radiomic_up[train_index], X_radiomic_up[test_index], Y_radiomic_up[train_index], Y_radiomic_up[test_index]
    classifier = LogisticRegression(random_state=42, solver='liblinear')
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))

# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[0],
        label=r'LR Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")

# save data
df_out = df_out.append({'Method': 'LR',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_lr = classifier

##############################  SVM  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_radiomic_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_radiomic_up[train_index], X_radiomic_up[test_index], Y_radiomic_up[train_index], Y_radiomic_up[test_index]
    classifier = SVC(random_state=42, **gs_svm.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[1],
        label=r'SVM Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out = df_out.append({'Method': 'SVM',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_svm = classifier

##############################  MLP  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_radiomic_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_radiomic_up[train_index], X_radiomic_up[test_index], Y_radiomic_up[train_index], Y_radiomic_up[test_index]
    classifier = MLP(random_state=42, **gs_mlp.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
    
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[5],
        label=r'MLP Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out = df_out.append({'Method': 'MLP',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_mlp = classifier

##############################  RF  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_radiomic_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_radiomic_up[train_index], X_radiomic_up[test_index], Y_radiomic_up[train_index], Y_radiomic_up[test_index]
    classifier = RandomForestClassifier(random_state=42, **gs_rf.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
    
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[2],
        label=r'RF Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out = df_out.append({'Method': 'RF',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_rf = classifier


# ##############################  HGBC  ####################################
# from sklearn.ensemble import HistGradientBoostingClassifier
# kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
# tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
# mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
# for train_index, test_index in kf.split(X=X_radiomic_up):
#     # 模型训练
#     Xtrain, Xtest, Ytrain, Ytest = X_radiomic_up[train_index], X_radiomic_up[test_index], Y_radiomic_up[train_index], Y_radiomic_up[test_index]
#     classifier = HistGradientBoostingClassifier(random_state=42, **gs_hgbc.best_params_)
#     classifier = classifier.fit(Xtrain, Ytrain)
#     # 统计指标
#     viz = plot_roc_curve(classifier, Xtest, Ytest)
#     interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
#     interp_tpr[0] = 0.0
#     tprs.append(interp_tpr)
#     aucs.append(viz.roc_auc)
#     accs.append(classifier.score(X=Xtest, y=Ytest))
#     TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
#     senss.append(TP / float(TP+FN))
#     specs.append(TN / float(TN+FP))
    
# # plot
# mean_tpr = np.mean(tprs, axis=0)
# mean_tpr[-1] = 1.0
# mean_auc = auc(mean_fpr, mean_tpr)
# std_auc = np.std(aucs)
# ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[3],
#         label=r'HGBC Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
#         lw=3, alpha=.8)
# ax.legend(loc="lower right")
# # save data
# df_out = df_out.append({'Method': 'HGBC',
#                         'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
#                         'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
#                         'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
#                         'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
# cls_hgbc = classifier

##############################  XGBoost  ####################################
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb

kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_radiomic_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_radiomic_up[train_index], X_radiomic_up[test_index], Y_radiomic_up[train_index], Y_radiomic_up[test_index]
    classifier =xgb.XGBClassifier(random_state=42, **gs_xgboost.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[4],
        label=r'XGBoost Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out = df_out.append({'Method': 'XGBoost',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_xgboost = classifier

##############################  Output  ###################################
# fig.savefig('methods.pdf')
# fig.savefig('methods.png', dpi=600)

In [ ]:
df_out

# 平扫、增强、合并模型

In [ ]:
##############################  Init Plot  ####################################
fig, ax = plt.subplots(figsize=(9,6))  # 用于绘制平均交叉验证
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey',
        label='Reference Line', alpha=.8)
ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
df_out2 = pd.DataFrame(columns=['Method','Acc' ,'AUC', 'Spec', 'Sens'])


##############################  RF(Preconcast Model)  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_pre_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_pre_up[train_index], X_pre_up[test_index], Y_pre_up[train_index], Y_pre_up[test_index]
    classifier = RandomForestClassifier(random_state=42, **gs_rf.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
    
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[1],
        label=r'Preconcast Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out2 = df_out2.append({'Method': 'Preconcast',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_pre = classifier

##############################  RF(posconcast Model)  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_pos_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_pos_up[train_index], X_pos_up[test_index], Y_pos_up[train_index], Y_pos_up[test_index]
    classifier = RandomForestClassifier(random_state=42, **gs_rf.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
    
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[0],
        label=r'Postconcast Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out2 = df_out2.append({'Method': 'Postconcast',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_pos = classifier

##############################  RF(Radiomic Model)  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_radiomic_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_radiomic_up[train_index], X_radiomic_up[test_index], Y_radiomic_up[train_index], Y_radiomic_up[test_index]
    classifier = RandomForestClassifier(random_state=42, **gs_rf.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
    
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[2],
        label=r'Radiomic Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out2 = df_out2.append({'Method': 'RF',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
# cls_radiomic = classifier

##############################  Output  ###################################
# fig.savefig('radiomic_compared.pdf')
# fig.savefig('radiomic_compared.png', dpi=600)

In [ ]:
df_out2

# 构建影像特征模型、临床模型、合并模型

In [ ]:
# 预处理临床数据特征

df_cli = pd.read_excel('./immunotherapy_112_V4.xlsx')
df_cli['Sex (Female=0,Male=1)'] = df_cli['Sex (Female=0,Male=1)'].map(lambda x: 'Female' if x==0 else 'Male')
df_cli['Smoking history（NO=0，YES=1）'] = df_cli['Smoking history（NO=0，YES=1）'].map(lambda x: 'No' if x==0 else 'Yes')
df_cli['Histologic type（Squamous cell carcinoma=0，nonsquamous cell carcinoma=1）'] = df_cli['Histologic type（Squamous cell carcinoma=0，nonsquamous cell carcinoma=1）'].map(lambda x: 'Squamous_cell_carcinoma' if x==0 else 'Nonsquamous_cell_carcinoma')
df_cli['Classification of Immunotherapy Drugs（PD-L1=0,PD-1=1）'] = df_cli['Classification of Immunotherapy Drugs（PD-L1=0,PD-1=1）'].map(lambda x: 'PD-L1' if x==0 else 'PD-1')
df_cli['Therapy line（1st=1，2nd=2，≥3rd=3）'] = df_cli['Therapy line（1st=1，2nd=2，≥3rd=3）'].map(lambda x: '1st' if x==1 else ('2nd' if x==2 else '3rd'))
df_cli['BMI category（<18.5=0,18.5-23.9=1,24-27.9=2,≥28=3）'] = df_cli['BMI category（<18.5=0,18.5-23.9=1,24-27.9=2,≥28=3）'].map(lambda x: '<18.5' if x==0 else ('18.5-23.9' if x==1 else ('24-27.9' if x==2 else '≥28')))
df_cli['Hemoglobin level （normal=0，decreased=1）'] = df_cli['Hemoglobin level （normal=0，decreased=1）'].map(lambda x: 'Normal' if x==0 else 'Decreased')
df_cli['Albumin level（normal=0，decreased=1）'] = df_cli['Albumin level（normal=0，decreased=1）'].map(lambda x: 'Normal' if x==0 else 'Decreased')
df_cli['LDH level（normal=0， increased =1）'] = df_cli['LDH level（normal=0， increased =1）'].map(lambda x: 'Normal' if x==0 else 'Increased')
df_cli['LIPI (good=0, intermediate=1, poor=2)'] = df_cli['LIPI (good=0, intermediate=1, poor=2)'].map(lambda x: 'good' if x==0 else ('Intermediate' if x==1 else 'poor'))
df_cli['tumor stage（IIIB=0,IIIC=1,IVA=2,IVB=3）'] = df_cli['tumor stage（IIIB=0,IIIC=1,IVA=2,IVB=3）'].map(lambda x: 'IIIB' if x==0 else ('IIIC' if x==1 else ('IVA' if x==2 else 'IVB')))
df_cli['COPD (NO=0，YES=1)'] = df_cli['COPD (NO=0，YES=1)'].map(lambda x: 'No' if x==0 else 'Yes')
df_cli['Pleural effusion(NO=0, YES=1)'] = df_cli['Pleural effusion(NO=0, YES=1)'].map(lambda x: 'No' if x==0 else 'Yes')
df_cli['Pericardial effusion(NO=0, YES=1)'] = df_cli['Pericardial effusion(NO=0, YES=1)'].map(lambda x: 'No' if x==0 else 'Yes')
df_cli['Bone metastasis (none=0, single=1, multiple=2)'] = df_cli['Bone metastasis (none=0, single=1, multiple=2)'].map(lambda x: 'None' if x==0 else ('Single' if x==1 else 'Multiple'))
df_cli['Brain  metastasis (none=0, single=1, multiple=2)'] = df_cli['Brain  metastasis (none=0, single=1, multiple=2)'].map(lambda x: 'None' if x==0 else ('Single' if x==1 else 'Multiple'))
df_cli['Liver metastasis (none=0, single=1, multiple=2)'] = df_cli['Liver metastasis (none=0, single=1, multiple=2)'].map(lambda x: 'None' if x==0 else ('Single' if x==1 else 'Multiple'))
df_cli.columns = ['patient_id',
                 '影像号',
                 'PFS(month)',
                 'PFS结局状态（0=删失，1=进展）',
                 'Label',
                 'Sex',
                 'Age',
                 'Smoking_history',
                 'ECOG score',
                 'Histologic_type',
                 'Classification_of_Immunotherapy_Drugs',
                 'Therapy_line',
                 'BMI',
                 'BMI_category',
                 'Hemoglobin',
                 'Hemoglobin_level',
                 'Albumin',
                 'Albumin_level',
                 'LDH',
                 'LDH_level',
                 'LIPI',
                 'dNLR',
                 'NLR',
                 'LMR',
                 'PLR',
                 'ALI',
                 'Tumor_stage',
                 'COPD',
                 'Bone_metastasis',
                 'Brain_metastasis',
                 'Liver_metastasis',
                 'Pleural_effusion',
                 'Pericardial_effusion']

df_cli = df_cli.iloc[:, 4:]
df_cli = pd.get_dummies(df_cli)
df_cli.Label = df_cli.Label.map(lambda x: 0 if x==1 else 1)
df_cli

# 数据扩充
X_cli = df_cli.iloc[:, 1:].values
Y_cli = df_cli.loc[:, 'Label']
ros = RandomOverSampler(random_state=42)
X_clin_up, Y_clin_up = ros.fit_resample(X_cli, Y_cli)

##############################  Init Plot  ####################################
fig, ax = plt.subplots(figsize=(9,6))  # 用于绘制平均交叉验证
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey',
        label='Reference Line', alpha=.8)
ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="ROC curve with multiple machine learning methods")
df_out1 = pd.DataFrame(columns=['Method','Acc' ,'AUC', 'Spec', 'Sens'])


##############################  RF (Clinical Model)  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_clin_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_clin_up[train_index], X_clin_up[test_index], Y_clin_up[train_index], Y_clin_up[test_index]
    classifier = RandomForestClassifier(random_state=42)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
    
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[0],
        label=r'Clinical Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")

cls_clin = classifier

In [ ]:
# # 数据预处理
# X_cli = df_merge.loc[:,features_cli].values
# Y_cli = df_merge.loc[:, 'Label_0 (DCB=0 PFS>6,NDB=1 PFS≤6)']

# ros = RandomOverSampler(random_state=42)
# X_clin_up, Y_clin_up = ros.fit_resample(X_cli, Y_cli)

In [ ]:
##############################  Init Plot  ####################################
fig, ax = plt.subplots(figsize=(9,6))  # 用于绘制平均交叉验证
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey',
        label='Reference Line', alpha=.8)
ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="ROC curve with multiple machine learning methods")
df_out1 = pd.DataFrame(columns=['Method','Acc' ,'AUC', 'Spec', 'Sens'])


# ##############################  RF (Clinical Model)  ####################################
# kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
# tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
# mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
# for train_index, test_index in kf.split(X=X_clin_up):
#     # 模型训练
#     Xtrain, Xtest, Ytrain, Ytest = X_clin_up[train_index], X_clin_up[test_index], Y_clin_up[train_index], Y_clin_up[test_index]
#     classifier = RandomForestClassifier(random_state=42)
#     classifier = classifier.fit(Xtrain, Ytrain)
#     # 统计指标
#     viz = plot_roc_curve(classifier, Xtest, Ytest)
#     interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
#     interp_tpr[0] = 0.0
#     tprs.append(interp_tpr)
#     aucs.append(viz.roc_auc)
#     accs.append(classifier.score(X=Xtest, y=Ytest))
#     TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
#     senss.append(TP / float(TP+FN))
#     specs.append(TN / float(TN+FP))
    
# # plot
# mean_tpr = np.mean(tprs, axis=0)
# mean_tpr[-1] = 1.0
# mean_auc = auc(mean_fpr, mean_tpr)
# std_auc = np.std(aucs)
# ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[0],
#         label=r'Clinical Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
#         lw=3, alpha=.8)
# ax.legend(loc="lower right")
# # save data
# df_out1 = df_out1.append({'Method': 'Clinical',
#                         'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
#                         'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
#                         'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
#                         'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
# cls_clin = classifier

##############################  RF(Radiomic Model)  ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_radiomic_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_radiomic_up[train_index], X_radiomic_up[test_index], Y_radiomic_up[train_index], Y_radiomic_up[test_index]
    classifier = RandomForestClassifier(random_state=42, **gs_rf.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
    
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[2],
        label=r'Radiomic Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out1 = df_out1.append({'Method': 'RF',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_radiomic = classifier

##############################  RF (Combined Model) ####################################
kf = KFold(n_splits=5, random_state=42, shuffle=True)  # 五折交叉验证
tprs, aucs, accs, specs, senss = [], [], [], [], []  # 初始化metric
mean_fpr = np.linspace(0, 1, 100)  # 初始化插值
for train_index, test_index in kf.split(X=X_radiomic_up):
    # 模型训练
    Xtrain, Xtest, Ytrain, Ytest = X_merge_up[train_index], X_merge_up[test_index], Y_merge_up[train_index], Y_merge_up[test_index]
    classifier = RandomForestClassifier(random_state=42, **gs_rf.best_params_)
    classifier = classifier.fit(Xtrain, Ytrain)
    # 统计指标
    viz = plot_roc_curve(classifier, Xtest, Ytest)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)  # 获取插值
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    accs.append(classifier.score(X=Xtest, y=Ytest))
    TN, FP, FN, TP = confusion_matrix(y_true=Ytest, y_pred=classifier.predict(Xtest)).ravel()
    senss.append(TP / float(TP+FN))
    specs.append(TN / float(TN+FP))
    
# plot
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color=sns.color_palette("Set2")[1],
        label=r'Combined Model (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=3, alpha=.8)
ax.legend(loc="lower right")
# save data
df_out1 = df_out1.append({'Method': 'Combined Model',
                        'Acc': str(round(np.mean(accs),3)) + "±" + str(round(np.std(accs),3)),
                        'AUC': str(round(mean_auc,3)) + "±" + str(round(std_auc,3)), 
                        'Spec':str(round(np.mean(specs),3)) + "±" + str(round(np.std(specs),3)), 
                        'Sens':str(round(np.mean(senss),3)) + "±" + str(round(np.std(senss),3))}, ignore_index=True)
cls_combined = classifier


##############################  Output  ###################################
# fig.savefig('data_compared.pdf')
# fig.savefig('data_compared.png', dpi=600)

In [ ]:
df_out1

In [ ]:
from sklearn.calibration import calibration_curve

# 修改字体
from matplotlib import rcParams
params={'font.family':'sans-serif',
        'font.sans-serif':'Arial',
        'font.weight':'normal',
        'font.size': 15
        }
rcParams.update(params)

fig, ax = plt.subplots(figsize=(8,6))  # 用于绘制平均交叉验证
n = 3
# clin
y_true = [0 if i== 1 else 1 for i in Y_clin_up]
y_prob = cls_clin.predict_proba(X_clin_up)[:,0]
prob_true, prob_pred = calibration_curve(y_true=y_true, y_prob=y_prob, n_bins=n)
plt.plot(prob_true, prob_pred, 'o-', c=sns.color_palette('Set2')[0], lw=3, alpha=.8, label='Clinical Model')

# radiomic
y_true = [0 if i== 1 else 1 for i in Y_radiomic_up]
y_prob = cls_radiomic.predict_proba(X_radiomic_up)[:,0]
prob_true, prob_pred = calibration_curve(y_true=y_true, y_prob=y_prob, n_bins=n)
plt.plot(prob_true, prob_pred, 'o-', c=sns.color_palette('Set2')[1], lw=3, alpha=.8, label='Radiomical Model')

# combined
y_true = [0 if i== 1 else 1 for i in Y_merge_up]
y_prob = cls_combined.predict_proba(X_merge_up)[:,0]
prob_true, prob_pred = calibration_curve(y_true=y_true, y_prob=y_prob, n_bins=n)
plt.plot(prob_true, prob_pred, 'o-', c=sns.color_palette('Set2')[2], lw=3, alpha=.8, label='Combined Model')

# ref line
plt.plot(np.linspace(0,1,5), np.linspace(0,1,5), '-', c='grey',lw=3, alpha=.8, label='Reference Line')


plt.legend()
plt.xlabel('Predicted DCB Rate')
plt.ylabel('Actual DCB Rate')

plt.savefig('calibration_DCB.pdf')
plt.savefig('calibration_DCB.png', dpi=600)

In [ ]:
from sklearn.calibration import calibration_curve

# 修改字体
from matplotlib import rcParams
params={'font.family':'sans-serif',
        'font.sans-serif':'Arial',
        'font.weight':'normal',
        'font.size': 15
        }
rcParams.update(params)

fig, ax = plt.subplots(figsize=(8,6))  # 用于绘制平均交叉验证

# clin
y_true = Y_clin_up
y_prob = cls_clin.predict_proba(X_clin_up)[:,1]
prob_true, prob_pred = calibration_curve(y_true=y_true, y_prob=y_prob, n_bins=3)
plt.plot(prob_true, prob_pred, 'o-', c=sns.color_palette('Set2')[0], lw=3, alpha=.8, label='Clinical Model')

# radiomic
y_true = Y_radiomic_up
y_prob = cls_radiomic.predict_proba(X_radiomic_up)[:,1]
prob_true, prob_pred = calibration_curve(y_true=y_true, y_prob=y_prob, n_bins=3)
plt.plot(prob_true, prob_pred, 'o-', c=sns.color_palette('Set2')[1], lw=3, alpha=.8, label='Radiomical Model')

# combined
y_true = Y_merge_up
y_prob = cls_combined.predict_proba(X_merge_up)[:,1]
prob_true, prob_pred = calibration_curve(y_true=y_true, y_prob=y_prob, n_bins=3)
plt.plot(prob_true, prob_pred, 'o-', c=sns.color_palette('Set2')[2], lw=3, alpha=.8, label='Combined Model')

# ref line
plt.plot(np.linspace(0,1,5), np.linspace(0,1,5), '-', c='grey',lw=3, alpha=.8, label='Reference Line')


plt.legend()
plt.xlabel('Predicted NDB Rate')
plt.ylabel('Actual NDB Rate')

plt.savefig('calibration.pdf')
plt.savefig('calibration.png', dpi=600)

# 临床特征相关性

In [ ]:
importances = cls_clin.feature_importances_
indices = np.argsort(importances)[::-1]
df_cli.columns[1:][indices]

In [ ]:
explainer = shap.TreeExplainer(cls_clin)
X = df_cli.iloc[:, 1:]
shap_values = explainer.shap_values(X)  # 传入特征矩阵X，计算SHAP值

In [ ]:
shap.summary_plot(shap_values[1], X, max_display=10)

In [ ]:
from pdpbox import pdp, get_dataset, info_plots

In [ ]:
base_features = X.columns.values.tolist()

feat_name = 'LDH'  # ca-num_major_vessels 原文
pdp_dist = pdp.pdp_isolate(
    model=cls_clin,  # 模型
    dataset=X,  # 测试集
    model_features=base_features,  # 特征变量；除去目标值 
    feature=feat_name  # 指定单个字段
)

pdp.pdp_plot(pdp_dist, feat_name)  # 传入两个参数
plt.show()

In [ ]:
base_features = X.columns.values.tolist()

feat_name = 'BMI'  # ca-num_major_vessels 原文
pdp_dist = pdp.pdp_isolate(
    model=cls_clin,  # 模型
    dataset=X,  # 测试集
    model_features=base_features,  # 特征变量；除去目标值 
    feature=feat_name  # 指定单个字段
)

pdp.pdp_plot(pdp_dist, feat_name)  # 传入两个参数
plt.show()

In [ ]:
base_features = X.columns.values.tolist()

feat_name = 'Hemoglobin'  # ca-num_major_vessels 原文
pdp_dist = pdp.pdp_isolate(
    model=cls_clin,  # 模型
    dataset=X,  # 测试集
    model_features=base_features,  # 特征变量；除去目标值 
    feature=feat_name  # 指定单个字段
)

pdp.pdp_plot(pdp_dist, feat_name)  # 传入两个参数
plt.show()

In [ ]:
base_features = X.columns.values.tolist()

feat_name = 'PLR'  # ca-num_major_vessels 原文
pdp_dist = pdp.pdp_isolate(
    model=cls_clin,  # 模型
    dataset=X,  # 测试集
    model_features=base_features,  # 特征变量；除去目标值 
    feature=feat_name  # 指定单个字段
)

pdp.pdp_plot(pdp_dist, feat_name)  # 传入两个参数
plt.show()

In [ ]:
base_features = X.columns.values.tolist()

feat_name = 'NLR'  # ca-num_major_vessels 原文
pdp_dist = pdp.pdp_isolate(
    model=cls_clin,  # 模型
    dataset=X,  # 测试集
    model_features=base_features,  # 特征变量；除去目标值 
    feature=feat_name  # 指定单个字段
)

pdp.pdp_plot(pdp_dist, feat_name)  # 传入两个参数
plt.show()

In [ ]:
base_features = X.columns.values.tolist()

feat_name = 'Albumin'  # ca-num_major_vessels 原文
pdp_dist = pdp.pdp_isolate(
    model=cls_clin,  # 模型
    dataset=X,  # 测试集
    model_features=base_features,  # 特征变量；除去目标值 
    feature=feat_name  # 指定单个字段
)

pdp.pdp_plot(pdp_dist, feat_name)  # 传入两个参数
plt.show()

In [ ]:
shap.summary_plot(shap_values, X, plot_type="bar")

# 生存曲线

In [ ]:
# 统计检验

from lifelines.statistics import logrank_test

logrank_test(durations_A=df_sur[df_sur.predict_label == 0]['PFS(month)'].values,
             durations_B=df_sur[df_sur.predict_label == 1]['PFS(month)'].values,
             event_observed_A=df_sur[df_sur.predict_label == 0]['PFS结局状态（0=删失，1=进展）'].values,
             event_observed_B=df_sur[df_sur.predict_label == 1]['PFS结局状态（0=删失，1=进展）'].values
            )

In [ ]:
import lifelines

In [ ]:
# 数据预处理

df_sur = df_cli.loc[:, ['Label_0 (DCB=0 PFS>6,NDB=1 PFS≤6)', 'PFS结局状态（0=删失，1=进展）', 'PFS(month)'] + list(features_cli)]
df_sur = df_sur.sort_index()
df_sur = df_sur.reset_index(drop=True)
df_sur = pd.concat([df_sur, df_rad_pos, df_rad_pre], axis=1)
df_sur['predict_label'] = cls_combined.predict(df_sur.iloc[:,3:].values)
df_sur

In [ ]:
# 生存曲线

from lifelines import KaplanMeierFitter

# 修改字体
from matplotlib import rcParams
params={'font.family':'sans-serif',
        'font.sans-serif':'Arial',
        'font.weight':'normal',
        'font.size': 15
        }
rcParams.update(params)

fig, ax = plt.subplots(figsize=(8,6))  # 用于绘制平均交叉验证

kmf = KaplanMeierFitter()
kmf.fit(df_sur[df_sur.predict_label == 0]['PFS(month)'].values, 
        df_sur[df_sur.predict_label == 0]['PFS结局状态（0=删失，1=进展）'].values,
        label='Predicted DCB')
ax = kmf.plot_survival_function(color=sns.color_palette('Set2')[0])
kmf.fit(df_sur[df_sur.predict_label == 1]['PFS(month)'].values, 
        df_sur[df_sur.predict_label == 1]['PFS结局状态（0=删失，1=进展）'].values,
        label='Predicted NDB')
ax = kmf.plot_survival_function(color=sns.color_palette('Set2')[1])
plt.xlabel('Time (Month)')

plt.savefig('KM.pdf')
plt.savefig('KM.png', dpi=600)